In [16]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [8]:
#!ls '/content/drive/MyDrive'

archive.zip


In [17]:
!unzip -q '/content/drive/MyDrive/archive.zip' -d '/content/face_dataset'

In [18]:
!pip install ultralytics

In [9]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageFilter
import numpy as np

In [24]:
dataset_path = '/content/face_dataset'

In [25]:
if not os.path.exists(dataset_path):
    print(f"Ошибка: Директория '{dataset_path}' не найдена.")
    print("Пожалуйста, убедитесь, что вы скачали и распаковали датасет по указанному пути.")


dataset_yaml_content = f"""train: {dataset_path}/images/train
val: {dataset_path}/images/val

nc: 1  # Количество классов (у нас только 'face')
names: ['face']  # Имена классов
"""

# Сохраняем содержимое в файл dataset.yaml
with open('dataset.yaml', 'w') as f:
    f.write(dataset_yaml_content)

print("Файл dataset.yaml создан.")

Файл dataset.yaml создан.


In [27]:
# Загрузка предобученной модели YOLOv8n (nano версия)
model = YOLO('yolov8n.pt')
print('Начало обучения модели YOLOv8')
results = model.train(
    data='dataset.yaml',
    epochs=20,
    batch=16,
    imgsz=640,
    lr0=0.01,
    optimizer='auto',
    weight_decay=0.0005,
    name='yolov8_face_detection',
    save_json=True,         # Сохраняет метрики в JSON (для mAP)
    save_hybrid=True,       # Сохраняет "гибридные" метрики
    plots=True,             # Генерирует графики (PR, Confusion Matrix)
)

#результаты
metrics = model.val()# Оценка на валидационном наборе
print(f"mAP50-95: {metrics.box.map}") # mAP от 0.5 до 0.95 IoU
print(f"mAP50: {metrics.box.map50}") # mAP при IoU=0.5

# Сохранение метрик в CSV
results_df = pd.read_csv("runs/detect/yolov8_face_detection/results.csv")
results_df.to_csv("training_metrics.csv", index=False)

Начало обучения модели YOLOv8
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_face_detection6, nbs=64, nms=False, opset=None, optimiz

train: Scanning /content/face_dataset/labels/train.cache... 13386 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13386/13386 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 336.3±239.0 MB/s, size: 308.9 KB)


val: Scanning /content/face_dataset/labels/val.cache... 3347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8_face_detection6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/yolov8_face_detection6
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:42<00:00,  2.45it/s]


                   all       3347      10299      0.894      0.807      0.879      0.587
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 2.8ms postprocess per image
Saving runs/detect/yolov8_face_detection6/predictions.json...
Results saved to runs/detect/yolov8_face_detection6
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1426.6±549.7 MB/s, size: 134.1 KB)


val: Scanning /content/face_dataset/labels/val.cache... 3347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 210/210 [00:44<00:00,  4.73it/s]


                   all       3347      10299      0.893      0.806      0.879      0.586
Speed: 0.4ms preprocess, 3.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving runs/detect/yolov8_face_detection62/predictions.json...
Results saved to runs/detect/yolov8_face_detection62
mAP50-95: 0.5862914612565557
mAP50: 0.8793359327226336


NameError: name 'pd' is not defined

In [4]:
import os
from ultralytics import YOLO
#детекция и блюр лиц
last_run_dir = '/home/sky/project 09-04'
best_model_path = os.path.join(last_run_dir, 'weights','best.pt')
trained_model = YOLO(best_model_path)
print(f'Загружена обученная модель:{best_model_path}')

Загружена обученная модель:/home/sky/project 09-04/weights/best.pt


In [11]:
import numpy as np
#функция, которая обнаруживает лица и применяет к ним блюр
def blur_faces(image_path, output_path, model):
    try:
        img_pil = Image.open(image_path).convert('RGB')
        img_np = np.array(img_pil)
        #выполняем детекцию
        results = model(img_np)
        #итерируемся по лицам, применяем блюр
        for r in results:
            for box in r.boxes:
                #координаты рамки
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                #выделение области лица
                face_region = img_pil.crop((x1, y1, x2, y2))
                blurred_face = face_region.filter(ImageFilter.GaussianBlur(radius=20))
                #вставка размытого лица обратно на изображение
                img_pil.paste(blurred_face, (x1, y1))
                #сохранение изображений
                img_pil.save(output_path)
                print(f'Изображение с блюром сохранено как:{output_path}')
    except FileNotFoundError:
        print(f'File not found:{image_path}')
    except Exception as e:
        print(f'Error:{e}')

In [15]:
if __name__ == "__main__":
    input_image_path = '/home/sky/project 09-04/blur/blur.png'  
    output_image_path = '/home/sky/project 09-04/blured.png'
    blur_faces(input_image_path, output_image_path, trained_model)


0: 448x640 7 faces, 48.7ms
Speed: 11.7ms preprocess, 48.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
Изображение с блюром сохранено как:/home/sky/project 09-04/blured.png
